In [1]:
!pip install torch
!pip install transformers


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer ,LEDForConditionalGeneration, LEDTokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

import pandas as pd
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict


c:\Users\Zhang Xiang\Desktop\Year 3\Sem 2\FYPJ\Project\RD_TextSummarizerForWebinar\Development\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Tokenize Data
def preprocess_function(sample):
    # tokenize inputs
    sample['input_ids'] = tokenizer(sample["transcript"], padding="max_length", truncation=True, return_tensors='pt').input_ids

    # Tokenize targets with the `text_target` keyword argument
    sample['labels'] = tokenizer(text_target=sample["summary"], padding="max_length", truncation=True, return_tensors='pt').input_ids

    
    return sample

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [5]:
df_train = pd.read_excel('../Data/tib_train.xlsx')
df_test = pd.read_excel('../Data/tib_test.xlsx')

df_train["id"] = df_train.index 
df_test["id"] = df_test.index + len(df_train)


# Convert DataFrames into Datasets
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# Create a DatasetDict to contain train and test splits
datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
})

# Access information about the combined dataset
for split_name, split_dataset in datasets.items():
    print(f"{split_name}: {split_dataset.info}")

print(datasets.keys())

train: DatasetInfo(description='', citation='', homepage='', license='', features={'summary': Value(dtype='string', id=None), 'transcript': Value(dtype='string', id=None), 'id': Value(dtype='int64', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)
test: DatasetInfo(description='', citation='', homepage='', license='', features={'summary': Value(dtype='string', id=None), 'transcript': Value(dtype='string', id=None), 'id': Value(dtype='int64', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)
dict_keys(['train', 'test'])


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = datasets.map(
        preprocess_function,
        batched=True,
    )


tokenized_dataset = tokenized_dataset.remove_columns(["transcript", "summary", "id"])

# for split in tokenized_dataset.keys():
    # tokenized_dataset[split] = tokenized_dataset[split].remove_columns(["transcript", "summary", "id"])

# tokenized_dataset = tokenized_dataset.remove_column(["transcript", "summary", "id"])
# tokenized_dataset = tokenized_dataset.rename_column= ["label", "labels"]
# tokenized_dataset = tokenized_dataset.with_format("torch")


# # save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("./data/train")
tokenized_dataset["test"].save_to_disk("./data/eval")


tokenized_dataset

Map: 100%|██████████| 35/35 [00:00<00:00, 61.96 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 241
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 35
    })
})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
    num_train_epochs=3,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=your_train_dataset,  # Replace with your training dataset
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
results = trainer.evaluate()
